In [1]:
!pip install timm
!pip install --upgrade wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.12.21
    Uninstalling wandb-0.12.21:
      Successfully uninstalled wandb-0.12.21
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.
allennlp 2.10.0 requires wandb<0.13.0,>=0.10.0, but you have wandb 0.13.5 which is incompatible.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import wandb
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedGroupKFold
import timm
from tqdm import tqdm
import albumentations as A
from collections import defaultdict
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score,precision_score,recall_score
from torch.cuda import amp
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import gc
import copy

In [3]:
class CFG:
    seed          = 42
    debug         = False # set debug=False for Full Training
    model         =  'tf_efficientnet_b0_ns'
    batch_size    = 32
    img_size      = [224, 224]
    epochs        = 10
    lr            = 1e-4
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-4
    T_max         = 449
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-6
    n_accumulate  = 1
    n_fold        = 5
    folds         = [0,1,2,3]
    device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
df = pd.read_csv('../input/old-ultra-sound-covid-detection/covid_data.csv')

def correct_path(idx):
    path = '../input/old-ultra-sound-covid-detection/data/'+idx[6::]
    return path

df['path'] = df['path'].apply(correct_path)

In [6]:
df['fold'] = -1
skf = StratifiedGroupKFold(n_splits=4,shuffle=True,random_state=42)
for fold,(train_idx,test_idx) in enumerate(skf.split(df['path'],df['label'],groups=df['video_id'])):
    df.loc[test_idx,'fold'] = fold

In [7]:
#df = df.head(1024)

In [8]:
df['video_id'].unique()

array(['Video_0', 'Video_1', 'Video_2', 'Video_3', 'Video_4', 'Video_5',
       'Video_6', 'Video_7', 'Video_8', 'Video_9', 'Video_10', 'Video_11',
       'Video_12', 'Video_13', 'Video_14', 'Video_15', 'Video_16',
       'Video_17', 'Video_18', 'Video_19', 'Video_20', 'Video_21',
       'Video_22', 'Video_23', 'Video_24', 'Video_25', 'Video_26',
       'Video_27', 'Video_28', 'Video_29', 'Video_30', 'Video_31',
       'Video_32', 'Video_33', 'Video_34', 'Video_35', 'Video_36',
       'Video_37', 'Video_38', 'Video_39', 'Video_40', 'Video_41',
       'Video_42', 'Video_43', 'Video_44', 'Video_45', 'Video_46',
       'Video_47', 'Video_48', 'Video_49', 'Video_50', 'Video_51',
       'Video_52', 'Video_53', 'Video_54', 'Video_55', 'Video_56',
       'Video_57', 'Video_58', 'Video_59', 'Video_60', 'Video_61',
       'Video_62', 'Video_63', 'Video_64', 'Video_65', 'Video_66',
       'Video_67', 'Video_68', 'Video_69', 'Video_70', 'Video_71',
       'Video_72', 'Video_73', 'Video_74', 'Vide

In [9]:
class CovidDataset(Dataset):    
    def __init__(self,df,transforms=None,is_valid=False):
        self.df = df
        self.transforms = transforms
        self.is_valid = is_valid
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img = cv2.imread(self.df['path'].iloc[idx])
        label = self.df['label'].iloc[idx]
        
        if self.transforms:
            img = self.transforms(image=img)['image']
        #img = torch.tensor(img,dtype=torch.float32)
        img = img.to(torch.float32)
        label = torch.tensor(label)
        
        if self.is_valid:
            return img,label,self.df['video_id'].iloc[idx]
        return img,label

In [10]:
def get_transforms(data):
    
    if data == 'train':
        return A.Compose([
            A.Resize(*CFG.img_size,interpolation=cv2.INTER_NEAREST),
            #A.RandomBrightnessContrast(),
            #A.CLAHE(),
            #A.GaussNoise(),
            #A.CenterCrop(*CFG.img_size),
            A.Normalize(),
            A.HorizontalFlip(),
            ToTensorV2(),
        ],p=1.0)

    elif data == 'valid':
        return A.Compose([
            A.Resize(*CFG.img_size,interpolation=cv2.INTER_NEAREST),
            #A.CenterCrop(*CFG.img_size),
            A.Normalize(),
            ToTensorV2(),
        ],p=1.0)

In [11]:
class BaseModel(nn.Module):
    def __init__(self,cfg,pretrained=False):
        super().__init__()
        self.cfg = cfg
        self.model = timm.create_model(cfg.model,pretrained=pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(self.n_features, 3)
        
    def forward(self,x):
        output = self.model(x)
        return output

In [12]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    epoch_loss = 0
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (images, labels) in pbar:
        labels = labels.type(torch.LongTensor)
        images = images.to(device, dtype=torch.float)
        labels  = labels.to(device)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            y_pred = model(images)
            loss   = criterion(y_pred, labels)
            loss   = loss / CFG.n_accumulate
            
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
    
        if (step + 1) % CFG.n_accumulate == 0:
            scaler.step(optimizer)
            scaler.update()
            # zero the parameter gradients
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        #print(epoch_loss)
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')
        torch.cuda.empty_cache()
        gc.collect()
    
    return epoch_loss

In [13]:
df['path'].iloc[0][47:]

'Video_0_frame_0.jpg'

In [14]:
def func(idx):
    return idx[47:]

In [15]:
df['unique_id'] = df['path'].apply(func)

In [16]:
df

,path,label,video_id,fold,unique_id
0,../input/old-ultra-sound-covid-detection/data/...,2,Video_0,2,Video_0_frame_0.jpg
1,../input/old-ultra-sound-covid-detection/data/...,2,Video_0,2,Video_0_frame_1.jpg
2,../input/old-ultra-sound-covid-detection/data/...,2,Video_0,2,Video_0_frame_2.jpg
3,../input/old-ultra-sound-covid-detection/data/...,2,Video_0,2,Video_0_frame_3.jpg
4,../input/old-ultra-sound-covid-detection/data/...,2,Video_0,2,Video_0_frame_4.jpg
...,...,...,...,...,...
19969,../input/old-ultra-sound-covid-detection/data/...,2,Video_131,1,Video_131_frame_70.jpg
19970,../input/old-ultra-sound-covid-detection/data/...,2,Video_131,1,Video_131_frame_71.jpg
19971,../input/old-ultra-sound-covid-detection/data/...,2,Video_131,1,Video_131_frame_72.jpg
19972,../input/old-ultra-sound-covid-detection/data/...,2,Video_131,1,Video_131_frame_73.jpg


In [17]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch,df,fold):
    label_save = np.array([])
    preds_save = np.array([])
    model.eval()
    epoch_loss = 0.0
    
    dataset_size = 0
    running_loss = 0.0
    
    
    val_scores = []
    video_ids_list = list(df[df['fold'] == fold]['video_id'].unique())
    unique_ids = list(df[df['fold']==fold]['unique_id'].values)
    video_ids_score = {}
    video_ids_count = {}
    for i in video_ids_list:
        video_ids_score[i] = 0
        
    for i in video_ids_list:
        df_1 = df[df['fold'] == fold].copy()
        video_ids_count[i] = len(df_1[df['video_id']==i])
        
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    oof = pd.DataFrame()
    scores = np.array([])
    real_labels = np.array([])
    real_ids = np.array([])
    for step, (images, labels, video_ids) in pbar:
        #images = images.type(torch.LongTensor)
        labels = labels.type(torch.LongTensor)
        images  = images.to(device, dtype=torch.float)
        labels   = labels.to(device)
        
        
        batch_size = images.size(0)
        
        y_pred  = model(images)
        #print(y_pred)
        #print(labels)
        loss    = criterion(y_pred, labels)
        
        
            
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        preds = y_pred.cpu().detach().numpy()
        preds = np.argmax(preds,axis=1).reshape(-1)
        #print(scores)
        scores = np.concatenate([scores,preds])
        #print(video_ids)
        real_labels = np.concatenate([real_labels,labels.cpu().numpy()])
        real_ids = np.concatenate([real_ids,video_ids])
        
        
        #y_pred = nn.Softmax()(y_pred)
        
        for i in range(len(video_ids)):
            #video_ids_count[video_ids[i]] +=1
            video_ids_score[video_ids[i]] += y_pred[i].cpu().detach().numpy()
            
        #val_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        #val_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        #val_scores.append([val_dice, val_jaccard])
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
    '''
    scores = []
    labels = []
    for ids in video_ids_score:
        video_ids_score[ids] = video_ids_score[ids]/video_ids_count[ids]
    
    for ids in video_ids_score:
        video_ids_score[ids] = np.argmax(video_ids_score[ids])
    
    for ids in video_ids_score:
        labels.append(df[df['video_id']==ids]['label'].iloc[0])
        scores.append(float(video_ids_score[ids]))
    scores = np.array(scores)
    '''
    #val_auc = roc_auc_score(labels,list(scores))
    
    labels = real_labels
    #print(labels.shape)
    #print(scores.shape)
    val_scores = f1_score(labels,scores, average='macro')
    val_accuracy = accuracy_score(labels,scores)
    precision = precision_score(labels,scores, average='macro')
    recall = recall_score(labels,scores, average='macro')
    video_ids_1 = []
    for ids in video_ids_score:
        video_ids_1.append(ids)
    #oof['video_id'] = video_ids_1
    #oof['preds'] = scores
    #oof['labels'] = labels
    torch.cuda.empty_cache()
    gc.collect()
    
    #print(len(unique_ids))
    #print(len(labels))
    
    df = pd.DataFrame({'id':unique_ids,'label':labels,'preds':scores})
    
    df.to_csv(f'oof_fold{fold}.csv',index=False)
    
    return epoch_loss, val_scores, val_accuracy, precision, recall

In [18]:
def run_training(model, optimizer, scheduler, device, num_epochs,df,fold):
    
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_score      = -np.inf
    best_epoch     = -1
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader,
                                     device=CFG.device, epoch=epoch)
        val_loss, val_scores, val_accuracy, precision, recall  = valid_one_epoch(model, valid_loader, 
                                                 device=CFG.device, 
                                                 epoch=epoch,df=df,fold=fold)
    
        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)
        history['Valid Score'].append(val_scores)
        
        
        #print(f'thresold: {best_th:0.4f}')
        print(f'Valid F1 Score: {val_scores:0.4f}')
        
        wandb.log({"Valid F1 Score": val_scores})
        
        print(f'Valid Accuracy Score: {val_accuracy:0.4f}')
        #print(f'Valid AUC Score: {val_auc:0.4f}')
        
        print(f'Valid Precision Score: {precision:0.4f}')
        print(f'Valid Recall Score: {recall:0.4f}')
        
        # deep copy the model
        if val_scores >= best_score:
            print(f"Valid Score Improved ({best_score:0.4f} ---> {val_scores:0.4f})")
            #best_oof = oof
            best_score    = val_scores
            best_epoch   = epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"best_epoch-{fold:02d}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved")
            
        last_model_wts = copy.deepcopy(model.state_dict())
        PATH = f"last_epoch-{fold:02d}.bin"
        torch.save(model.state_dict(), PATH)
            
        print(); print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    #best_oof.to_csv(f'{fold}_oof.csv',index=False)
    print("Best Score: {:.4f}".format(best_score))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [19]:
def fetch_scheduler(optimizer):
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CFG.T_max, 
                                                   eta_min=CFG.min_lr)
    elif CFG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG.T_0, 
                                                             eta_min=CFG.min_lr)
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=0.1,
                                                   patience=7,
                                                   threshold=0.0001,
                                                   min_lr=CFG.min_lr,)
    elif CFG.scheduler == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif CFG.scheduler == 'None':
        #scheduler = None
        return None
        
    return scheduler

In [20]:
criterion = nn.CrossEntropyLoss()

In [21]:
CONFIG = dict(
    seed = 42,
    model_name = 'tf_efficientnet_b0_ns',
    train_batch_size = 32,
    valid_batch_size = 32,
    img_size = 224,
    epochs = 10,
    learning_rate = 1e-4,
    scheduler     = 'CosineAnnealingLR',
    min_lr        = 1e-4,
    T_max         = 449,
    T_0           = 25,
    warmup_epochs = 0,
    wd            = 1e-6,
    n_accumulate  = 1,
    n_fold        = 5,
    folds         = [0, 1, 2, 3],
    #weight_decay = 1e-6,
    num_classes = 3,
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    competition = 'UltraSound',
    _wandb_kernel = 'deb'
)

In [22]:
run = wandb.init(project='UltraSound', 
                 config=CONFIG,
                 job_type='Train',
                 anonymous='must')

wandb: Currently logged in as: mrigendra. Use `wandb login --relogin` to force relogin


In [23]:
for fold in CFG.folds:
    print(f'#'*15)
    print(f'### Fold: {fold}')
    print(f'#'*15)
    train_df = df[df['fold'] !=fold].reset_index(drop=True)
    valid_df = df[df['fold'] ==fold].reset_index(drop=True)
    if CFG.debug:
        train_df = train_df.head()
        valid_df = valid_df.head()
        
    #train_df = train_df.head()

    
    train_dataset = CovidDataset(train_df,transforms=get_transforms(data='train'),is_valid=False)
    valid_dataset = CovidDataset(valid_df,transforms=get_transforms(data='valid'),is_valid=True)
    
    train_loader = DataLoader(train_dataset,batch_size=CFG.batch_size,shuffle=True,num_workers=2,drop_last=True)
    valid_loader = DataLoader(valid_dataset,batch_size=CFG.batch_size,shuffle=False,num_workers=2,drop_last=False)
    
    cfg = CFG()
    
    model     = BaseModel(cfg,pretrained=True)
    model.to(CFG.device)
    optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
    scheduler = fetch_scheduler(optimizer)
    model, history = run_training(model, optimizer, scheduler,
                                  device=CFG.device,
                                  num_epochs=CFG.epochs,
                                 df=df,
                                 fold=fold)

###############
### Fold: 0
###############


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b0_ns-c0e6a31c.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b0_ns-c0e6a31c.pth


cuda: Tesla P100-PCIE-16GB

Epoch 1/10

Train : 100%|██████████| 449/449 [03:21<00:00,  2.22it/s, gpu_mem=1.76 GB, lr=0.00010, train_loss=0.4235]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
Valid : 100%|██████████| 175/175 [00:44<00:00,  3.95it/s, gpu_memory=0.81 GB, lr=0.00010, valid_loss=0.2151]


Valid F1 Score: 0.9312
Valid Accuracy Score: 0.9333
Valid Precision Score: 0.9470
Valid Recall Score: 0.9214
Valid Score Improved (-inf ---> 0.9312)
Model Saved


Epoch 2/10

...

Valid F1 Score: 0.8369
Valid Accuracy Score: 0.8642
Valid Precision Score: 0.8617
Valid Recall Score: 0.8351


Epoch 9/10

Train : 100%|██████████| 504/504 [03:23<00:00,  2.47it/s, gpu_mem=1.78 GB, lr=0.00010, train_loss=0.0255]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
Valid : 100%|██████████| 120/120 [00:17<00:00,  6.82it/s, gpu_memory=0.80 GB, lr=0.00010, valid_loss=1.8891]


Valid F1 Score: 0.8139
Valid Accuracy Score: 0.8519
Valid Precision Score: 0.8098
Valid Recall Score: 0.8242


Epoch 10/10

Train : 100%|██████████| 504/504 [03:26<00:00,  2.44it/s, gpu_mem=1.78 GB, lr=0.00010, train_loss=0.0038]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
Valid : 100%|██████████| 120/120 [00:17<00:00,  6.67it/s, gpu_memory=0.80 GB, lr=0.00010, valid_loss=1.5328]


Valid F1 Score: 0.8045
Valid Accuracy Score: 0.8852
Valid Precision Score: 0.9213
Valid Recall Score: 0.7822


Training complete in 0h 37m 6s
Best Score: 0.8842


In [24]:
pd.read_csv('/kaggle/working/oof_fold0.csv')

,id,label,preds
0,Video_1_frame_0.jpg,2.0,0.0
1,Video_1_frame_1.jpg,2.0,0.0
2,Video_1_frame_2.jpg,2.0,0.0
3,Video_1_frame_3.jpg,2.0,2.0
4,Video_1_frame_4.jpg,2.0,0.0
...,...,...,...
5584,Video_130_frame_101.jpg,2.0,1.0
5585,Video_130_frame_102.jpg,2.0,1.0
5586,Video_130_frame_103.jpg,2.0,1.0
5587,Video_130_frame_104.jpg,2.0,1.0


In [25]:
run.finish()

Valid F1 Score,█▇▅▇▅▆▇▇▇▇▇▆█▂▇▃▆▇▇▄▃▄▄▃▄▃▄▃▄▄▂▁▂▅▄▁▅▃▂▁
Valid F1 Score,0.80448
